# Feathr Feature Store on Azure Demo Notebook

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

## Model Problem Statement
The e-commerce website has collected past user ratings for various products. The websie also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

After the model is trained, given a user_id, product_id pair and features, we should be able to predict the product rating that the user will give for this product_id.

(Compared with [the beginner version of product recommendation](product_recommendation_demo.ipynb), this tutorial expanded the example by predicting ratings for all products.)

## Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by user_id and product_id. So there might be 3 types of features:
* User features that are different for different users but are the same for different products. For example, user age is different for different users but it's the same for all products(or it's product-agnostic).
* Product features that are different for different products but are the same for different users.
* User-to-product features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

We will focus on the first two in our example.

The feature creation flow is as below:
![Feature Flow](https://github.com/linkedin/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## Prerequisite: Use Quick Start Template to Provision Azure Resources

First step is to provision required cloud resources if you want to use Feathr. Feathr provides a python based client to interact with cloud resources.

Please follow the steps [here](https://linkedin.github.io/feathr/how-to-guides/azure-deployment-arm.html) to provision required cloud resources. Due to the complexity of the possible cloud environment, it is almost impossible to create a script that works for all the use cases. Because of this, [azure_resource_provision.sh](https://github.com/linkedin/feathr/blob/main/docs/how-to-guides/azure_resource_provision.sh) is a full end to end command line to create all the required resources, and you can tailor the script as needed, while [the companion documentation](https://linkedin.github.io/feathr/how-to-guides/azure-deployment-cli.html) can be used as a complete guide for using that shell script. 


![Architecture](https://github.com/linkedin/feathr/blob/main/docs/images/architecture.png?raw=true)

## Prerequisite: Install Feathr 

Install Feathr using pip:

`pip install -U feathr pandavro scikit-learn`

## Prerequisite: Configure the required environment with Feathr Quick Start Template

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. Run the code below to install Feathr, login to Azure to get the required credentials to access more cloud resources.

**REQUIRED STEP: Fill in the resource prefix when provisioning the resources**

In [ ]:
resource_prefix = "ckim2"

In [ ]:
! pip install feathr azure-cli  pandavro scikit-learn


Login to Azure with a device code (You will see instructions in the output):

In [ ]:
! az login --use-device-code

In [ ]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient


**Permission**

To proceed with the following steps, you may need additional permission: permission to access the keyvault, permission to access the Storage Blob as a Contributor and permission to submit jobs to Synapse cluster. Skip this step if you have already given yourself the access. Otherwise, run the following lines of command in the Cloud Shell before running the cell below.

~~~ 
userId=<email_id_of_account_requesting_access>
resource_prefix=<resource_prefix>
synapse_workspace_name="${resource_prefix}syws"
keyvault_name="${resource_prefix}kv"
objectId=$(az ad user show --id $userId --query id -o tsv)
az keyvault update --name $keyvault_name --enable-rbac-authorization false
az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
~~~

**Get all the required credentials from Azure KeyVault**

In [ ]:
# Get all the required credentials from Azure Key Vault
key_vault_name=resource_prefix+"kv"
synapse_workspace_url=resource_prefix+"syws"
adls_account=resource_prefix+"dls"
adls_fs_name=resource_prefix+"fs"
purview_name=resource_prefix+"purview"
key_vault_uri = f"https://{key_vault_name}.vault.azure.net"
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
client = SecretClient(vault_url=key_vault_uri, credential=credential)
secretName = "FEATHR-ONLINE-STORE-CONN"
retrieved_secret = client.get_secret(secretName).value

# Get redis credentials; This is to parse Redis connection string.
redis_port=retrieved_secret.split(',')[0].split(":")[1]
redis_host=retrieved_secret.split(',')[0].split(":")[0]
redis_password=retrieved_secret.split(',')[1].split("password=",1)[1]
redis_ssl=retrieved_secret.split(',')[2].split("ssl=",1)[1]

# Set the resource link
os.environ['spark_config__azure_synapse__dev_url'] = f'https://{synapse_workspace_url}.dev.azuresynapse.net'
os.environ['spark_config__azure_synapse__pool_name'] = 'spark31'
os.environ['spark_config__azure_synapse__workspace_dir'] = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_project'
os.environ['feature_registry__purview__purview_name'] = f'{purview_name}'
os.environ['online_store__redis__host'] = redis_host
os.environ['online_store__redis__port'] = redis_port
os.environ['online_store__redis__ssl_enabled'] = redis_ssl
os.environ['REDIS_PASSWORD']=redis_password
os.environ['feature_registry__purview__purview_name'] = f'{purview_name}'
feathr_output_path = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_output'

## Prerequisite: Configure the required environment (Don't need to update if using the above Quick Start Template)

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. If you use Feathr CLI to create a workspace, you should have a folder with a file called `feathr_config.yaml` in it with all the required configurations. Otherwise, update the configuration below.

The code below will write this configuration string to a temporary location and load it to Feathr. Please still refer to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It should also have more explanations on the meaning of each variable.

In [ ]:
import tempfile
yaml_config = """
# Please refer to https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml for explanations on the meaning of each field.
api_version: 1
project_config:
  project_name: 'feathr_getting_started'
  required_environment_variables:
    - 'REDIS_PASSWORD'
offline_store:
  adls:
    adls_enabled: true
  wasb:
    wasb_enabled: true
  s3:
    s3_enabled: false
    s3_endpoint: 's3.amazonaws.com'
  jdbc:
    jdbc_enabled: false
    jdbc_database: 'feathrtestdb'
    jdbc_table: 'feathrtesttable'
  snowflake:
    url: "dqllago-ol19457.snowflakecomputing.com"
    user: "feathrintegration"
    role: "ACCOUNTADMIN"
spark_config:
  spark_cluster: 'azure_synapse'
  spark_result_output_parts: '1'
  azure_synapse:
    dev_url: 'https://feathrazuretest3synapse.dev.azuresynapse.net'
    pool_name: 'spark3'
    workspace_dir: 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started'
    executor_size: 'Small'
    executor_num: 4
    feathr_runtime_location: wasbs://public@azurefeathrstorage.blob.core.windows.net/feathr-assembly-LATEST.jar
  databricks:
    workspace_instance_url: 'https://adb-2474129336842816.16.azuredatabricks.net'
    config_template: {'run_name':'','new_cluster':{'spark_version':'9.1.x-scala2.12','node_type_id':'Standard_D3_v2','num_workers':2,'spark_conf':{}},'libraries':[{'jar':''}],'spark_jar_task':{'main_class_name':'','parameters':['']}}
    work_dir: 'dbfs:/feathr_getting_started'
    feathr_runtime_location: https://azurefeathrstorage.blob.core.windows.net/public/feathr-assembly-LATEST.jar
online_store:
  redis:
    host: 'feathrazuretest3redis.redis.cache.windows.net'
    port: 6380
    ssl_enabled: True
feature_registry:
  api_endpoint: "https://feathr-sql-registry.azurewebsites.net/api/v1"
"""
tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)


## Setup necessary environment variables (Skip if using the above Quick Start Template)

You should setup the environment variables in order to run this sample. More environment variables can be set by referring to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It also has more explanations on the meaning of each variable.

To run this notebook, for Azure users, you need AZURE_CLIENT_ID, AZURE_TENANT_ID, AZURE_CLIENT_SECRET and REDIS_PASSWORD.
To run this notebook, for Databricks useres, you need DATABRICKS_WORKSPACE_TOKEN_VALUE and REDIS_PASSWORD.

# Initialize Feathr Client

In [ ]:
client = FeathrClient(config_path=tmp.name)

## Explore the raw source data
We have 4 datasets to work with: one observation dataset(a.k.a. label dataset), two raw datasets to generate features for users, one raw datasets to generate features for product.

In [ ]:
# Observation dataset(a.k.a. label dataset)
# Observation dataset usually comes with a event_timestamp to denote when the observation happened.
# The label here is product_rating. Our model objective is to predict a user's rating for this product.
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_observation_mock_data.csv")

In [ ]:
# User profile dataset
# Used to generate user features
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_profile_mock_data.csv")

In [ ]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to genearte features.
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_purchase_history_mock_data.csv")

In [ ]:
# Product detail dataset.
# Used to generate product features.
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/product_detail_mock_data.csv")

## Defining Features with Feathr
Let's try to create features from those raw source data.
In Feathr, a feature is viewed as a function, mapping from entity id or key, and timestamp to a feature value. For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://github.com/linkedin/feathr/blob/main/docs/concepts/feature-definition.md)


1. The typed key (a.k.a. entity key) identifies the subject of feature, e.g. a user id, 123.
2. The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
3. The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.
4. The timestamp indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

#### Define Anchors and Features
A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

#### Feature source
A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

In [ ]:
from pyspark.sql import SparkSession, DataFrame
def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col
    df = df.withColumn("tax_rate_decimal", col("tax_rate")/100)
    df.show(10)
    return df

batch_source = HdfsSource(name="userProfileData",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_profile_mock_data.csv",
                          preprocessing=feathr_udf_preprocessing)

In [ ]:
# Let's define some features for users so our recommendation can be customized for users.
user_id = TypedKey(key_column="user_id",
                       key_column_type=ValueType.INT32,
                       description="user id",
                       full_name="product_recommendation.user_id")

feature_user_age = Feature(name="feature_user_age",
                           key=user_id,
                           feature_type=INT32, transform="age")
feature_user_tax_rate = Feature(name="feature_user_tax_rate",
                                key=user_id,
                                feature_type=FLOAT,
                                transform="tax_rate_decimal")
feature_user_gift_card_balance = Feature(name="feature_user_gift_card_balance",
                                    key=user_id,
                                    feature_type=FLOAT,
                                    transform="gift_card_balance")
feature_user_has_valid_credit_card = Feature(name="feature_user_has_valid_credit_card",
                                    key=user_id,
                                    feature_type=BOOLEAN,
                                    transform="number_of_credit_cards > 0")
                                    
features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card
]

user_feature_anchor = FeatureAnchor(name="anchored_features",
                               source=batch_source,
                               features=features)

In [ ]:
# Let's define some features for the products so our recommendation can be customized for proudcts.
product_batch_source = HdfsSource(name="productProfileData",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/product_detail_mock_data.csv")

product_id = TypedKey(key_column="product_id",
                       key_column_type=ValueType.INT32,
                       description="product id",
                       full_name="product_recommendation.product_id")

feature_product_quantity = Feature(name="feature_product_quantity",
                           key=product_id,
                           feature_type=FLOAT, 
                           transform="quantity")
feature_product_price = Feature(name="feature_product_price",
                                key=product_id,
                                feature_type=FLOAT,
                                transform="price")
                                    
product_features = [
    feature_product_quantity,
    feature_product_price
]

product_anchor = FeatureAnchor(name="product_anchored_features",
                               source=product_batch_source,
                               features=product_features)

### Window aggregation features

For window aggregation features, see the supported fields below:

Note that the `agg_func` should be any of these:

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |


After you have defined features and sources, bring them together to build an anchor:


Note that if the data source is from the observation data, the `source` section should be `INPUT_CONTEXT` to indicate the source of those defined anchors.

In [ ]:
purchase_history_data = HdfsSource(name="purchase_history_data",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_purchase_history_mock_data.csv",
                          event_timestamp_column="purchase_date",
                          timestamp_format="yyyy-MM-dd")
                          
agg_features = [Feature(name="feature_user_totla_purchase_in_90days",
                        key=user_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(purchase_amount)",
                                                          agg_func="AVG",
                                                          window="90d"))
                ]

user_agg_feature_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=purchase_history_data,
                           features=agg_features)

### Derived Features Section
Derived features are the features that are computed from other features. They could be computed from anchored features, or other derived features.

In [ ]:
feature_user_purchasing_power = DerivedFeature(name="feature_user_purchasing_power",
                                      key=user_id,
                                      feature_type=FLOAT,
                                      input_features=[
                                          feature_user_gift_card_balance, feature_user_has_valid_credit_card],
                                      transform="feature_user_gift_card_balance + if_else(toBoolean(feature_user_has_valid_credit_card), 100, 0)")

And then we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features (which is not anchored to a source).

In [ ]:
client.build_features(anchor_list=[user_agg_feature_anchor, user_feature_anchor, product_anchor], derived_feature_list=[
                      feature_user_purchasing_power])

## Create training data using point-in-time correct feature join

A training dataset usually contains entity id columns, multiple feature columns, event timestamp column and label/target column. 

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. 

To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/linkedin/feathr/blob/main/docs/concepts/point-in-time-join.md)

In [ ]:
if client.spark_runtime == 'databricks':
    output_path = 'dbfs:/feathrazure_test.avro'
else:
#     output_path = 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/test123_temp/product_rec_new'
    output_path = feathr_output_path


user_feature_query = FeatureQuery(
    feature_list=["feature_user_age", 
                  "feature_user_tax_rate", 
                  "feature_user_gift_card_balance", 
                  "feature_user_has_valid_credit_card", 
                  "feature_user_totla_purchase_in_90days",
                  "feature_user_purchasing_power"
                  ], 
    key=user_id)

product_feature_query = FeatureQuery(
    feature_list=[
                  "feature_product_quantity",
                  "feature_product_price"
                  ], 
    key=product_id)

settings = ObservationSettings(
    observation_path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_observation_mock_data.csv",
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd")
client.get_offline_features(observation_settings=settings,
                            feature_query=[user_feature_query, product_feature_query],
                            output_path=output_path)
client.wait_job_to_finish(timeout_sec=1000)

## Download the training dataset and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [ ]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    client.feathr_spark_launcher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(client)

df_res

## Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
final_df = df_res

final_df.drop(["event_timestamp"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['product_rating'] = final_df['product_rating'].astype("float64")

train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["product_rating"], axis=1),
                                                    final_df["product_rating"],
                                                    test_size=0.2,
                                                    random_state=42)
model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


## Materialize feature value into offline/online storage

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We can push the generated features to the online store like below:

In [ ]:
# Materialize user features
# (You can only materialize features of same entity key into one table so we can only materialize user features first.)
backfill_time = BackfillTime(start=datetime(
    2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings("user_feature_setting",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["feature_user_age", "feature_user_gift_card_balance"])

client.materialize_features(settings)
client.wait_job_to_finish(timeout_sec=1000)

We can then get the features from the online store (Redis):

## Fetching feature value for online inference

For features that are already materialized by the previous step, their latest value can be queried via the client's
`get_online_features` or `multi_get_online_features` API.

In [ ]:
client.get_online_features('user_features', '2', [
                                 'feature_user_age', 'feature_user_gift_card_balance'])

In [ ]:
client.multi_get_online_features('user_features', ['1', '2'], [
                                 'feature_user_age', 'feature_user_gift_card_balance'])


## Materialize product features

We can also materialize product features into a separate table.

In [ ]:
# Materialize product features
backfill_time = BackfillTime(start=datetime(
    2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))
redisSink = RedisSink(table_name="product_features")
settings = MaterializationSettings("product_feature_setting",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["feature_product_price"])

client.materialize_features(settings)
client.wait_job_to_finish(timeout_sec=1000)

In [ ]:
client.get_online_features('product_feature_setting', '2', [
                                 'feature_product_price'])

client.get_online_features('product_features', '2', [
                                 'feature_product_price'])

### Registering and Fetching features

We can also register the features with an Apache Atlas compatible service, such as Azure Purview, and share the registered features across teams:

In [ ]:
client.register_features()
client.list_registered_features(project_name="feathr_getting_started")